In [1]:
import os
import requests
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw
from io import BytesIO
from paddleocr import PaddleOCR

c:\Users\aashi\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
# Initialize PaddleOCR (you only need to do this once)
ocr = PaddleOCR(use_angle_cls=True, lang='en') 

# Path to your CSV file
csv_file_path = r'student_resource3\dataset\train.csv'

# Output folder for the OCR processed images and CSV
output_folder = r'E:\Career\hackathon\Amazon-ML-Challenge\paddle_ocr_output'
os.makedirs(output_folder, exist_ok=True)

# Read the CSV (limit to 100 rows for this example)
df = pd.read_csv(csv_file_path).head(20000)

[2024/09/15 22:33:44] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\aashi/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\aashi/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=

In [3]:
# Function to process each row
def process_row(row):
    img_url = row['image_link']
    group_id = row['group_id']
    entity_name = row['entity_name']
    entity_value = row['entity_value']

    ocr_result_text = ""

    try:
        # Download the image from the URL
        response = requests.get(img_url)
        if response.status_code == 200:
            img = Image.open(BytesIO(response.content)).convert('RGB')

            # Convert the image to a NumPy array
            img_np = np.array(img)

            # Perform OCR on the image
            result = ocr.ocr(img_np, cls=True)

            if result is not None:
                # Concatenate all detected texts
                for res in result:
                    if res is not None:
                        for line in res:
                            text = line[1][0]  # Extract the text only
                            ocr_result_text += text + " "  # Concatenate the text
    except Exception as e:
        print(f"Error processing image {img_url}: {e}")
    
    # Return the result as a dictionary for each row
    return {
        'group_id': group_id,
        'text': ocr_result_text.strip(),  # Clean up trailing spaces
        'entity_name': entity_name,
        'entity_value': entity_value
    }

In [ ]:
processed_data = df.apply(process_row, axis=1)

# Break the result into smaller chunks to handle failures
chunk_size = 20000  # Adjust based on your preference
for i in range(0, len(processed_data), chunk_size):
    chunk = processed_data.iloc[i:i+chunk_size]
    output_csv_path = os.path.join(output_folder, f'ocr_results_chunk_{i // chunk_size}.csv')
    chunk.to_csv(output_csv_path, index=False)
    print(f"Chunk saved to {output_csv_path}")

print("OCR processing complete.")
